# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [ ]:
import pandas as pd

airbnb = pd.read_csv("/Users/vaibhavjha/wrangling/assignment/data/airbnb_hw.csv")

airbnb['Price'] = airbnb['Price'].str.replace(',', '') # REPLACE commas with nothing
print(airbnb['Price'].unique()) # Gives a list of all the unique values

airbnb['Price'] = pd.to_numeric(airbnb['Price'], errors='coerce') # COERCE column to numeric
print(airbnb['Price'])

airbnb['Price'].isnull().sum() # 0 missing values

In [ ]:
import pandas as pd
import numpy as np

sharks = pd.read_csv("/Users/vaibhavjha/wrangling/assignment/data/sharks.csv", low_memory = False)
print(sharks.head(10))
sharks['Type'].unique()
sharks['Type'].isnull().sum()

# Missing data on 4087: no data in any column
print(sharks[sharks['Type'].isnull()])
sharks['Reason Missing'] = None
sharks.loc[4087,'Reason Missing'] = 'No data whatsoever on person'
print(sharks.loc[4087,:])

# Missing data on 239, 537 (unreported)
sharks.loc[(239, 537),'Reason Missing'] = 'Unreported'
print(sharks.loc[(239,537),:])

# Missing data on 5025, 5864 (reported)
sharks.loc[(5025, 5864),'Reason Missing'] = 'Reported'
print(sharks.loc[(5025,5864),:])


# Stratified the 'Type' missing observations as: No data whatsoever, unreported, and reported

In [ ]:
print(sharks['Type'].value_counts()) # Gives different types and frequencies

type = sharks['Type']
type = type.replace(['Sea Disaster', 'Watercraft', 'Boat', 'Boating', 'Boatomg'], 'Water issues') # Grouped together all known water issues
type.value_counts()

type = type.replace(['Invalid', 'Questionable', 'Unconfirmed', 'Unverified', 'Under investigation'], np.nan) # Removed all values associated with these types
type.value_counts()

sharks['Type'] = type # Replace the column in the sharks df
sharks['Type'].value_counts() # Complete, cleaned categorical variable

In [ ]:
import pandas as pd
import numpy as np

crime = pd.read_parquet("/Users/vaibhavjha/wrangling/assignment/data/justice_data.parquet")
crime.head(10)

print(crime['WhetherDefendantWasReleasedPretrial'])
crime = crime.rename(columns = {'WhetherDefendantWasReleasedPretrial':'Pretrial'}) # Shortened column name

crime['Pretrial'].unique()
crime['Pretrial'].head(10)
crime['Pretrial'].value_counts()

# It seems the parquet file codes the variables in 0 and 1, and that 9's are missing values.
# Want to recode 9's as np.nan's

crime['Pretrial'] = crime['Pretrial'].replace([9], np.nan) # Notice the list
print(crime['Pretrial'])
print(crime['Pretrial'].value_counts()) 

# Now have 1s and 0s, with missing values as np.nan

In [23]:
crime = crime.rename(columns = {'ImposedSentenceAllChargeInContactEvent':'Contact'})
crime['Contact'].unique()
crime['Contact'].value_counts()
# it seems the most popular value is a blank value
crime['Contact'] = pd.to_numeric(crime['Contact'], errors='coerce') # made the column go from string to numeric, doing so marked blanks as NaN
crime['Contact'] = crime['Contact'].round(0)
crime['Contact'].head(20)

array([9, 0, 1, 4, 2])

**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?